<a href="https://colab.research.google.com/github/AviSimkin/Misc.-stuff/blob/main/Cal_xls_expense_file_to_aggregate_by_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

note that you might need to install pandas 1.2.0 or newer to load the .xls file

In [110]:
###################################
### Loading file to a pandas df ###
###################################

import pandas as pd

expenses_file_path = '/content/drive/MyDrive/CreditCardPayments/Transactions_03_09_2022.xls'

# First we need to count number of rows to skip the last one
with open(expenses_file_path, 'r', encoding='utf-16') as fp:
    last_row = len(fp.readlines())-1


# Using read_csv is okay even though it's an xls file, don't worry
expenses_df = pd.read_csv(expenses_file_path, sep='\t', skiprows=[0,1,last_row], encoding='utf-16')
expenses_df.head()

,תאריך העסקה,שם בית העסק,סכום העסקה,סכום החיוב,פירוט נוסף
0,29/07/22,AMZN MKTP DE AMAZON.DE,€ -2.18,₪ -7.57,זיכוי בגין רכישה רגילה*
1,01/08/22,GENE BY GENE/FAMILY TREE,$ 58.95,₪ 200.47,"עסקה עם הנחה בעמלת עסקאות מט""ח*"
2,01/08/22,PAYPAL *ONETREEPLAN,₪ 350.11,₪ 350.11,NaN
3,02/08/22,מנהרות הכרמל-הוק,₪ 42.59,₪ 42.59,הוראת קבע רכישה רגילה*
4,02/08/22,שפע זול,₪ 48.41,₪ 48.41,NaN


In [111]:
############################################################################
### Creating a dictionary with business name keyword mapping to category ###
############################################################################

# Writing keywords of different business names for each category
category2shortbiz = {"Food and Groceris": ["שפע זול", "מרכז המזון","תן ביס","מאפיית","מש - קר","לואיזה", "גפניקה","שופרסל","Wolt"],
                   "Donations": ["YLDNEIRF NAGEV", "*DONATION", "ONETREE"],
                   "Internet Shopping": ["AMZN","Amazon","הייטקזון"],
                   "Bills" : ["NETFLIX","YouTubePremium","חברת החשמל","קרן מכבי","מרפאת שיניים","טלזר","אקספון","HOT"],
                   "Transportation" : ["רב-פס","YELLOW","NAD ELBBUB","מנהרות"],
                   "P2P payments": ["PAYBOX"],
                   "Other": ["הטכניון","אלי לבן","מקס סטוק","סופר פארם","GENE BY GENE","קונים בונים"],
                   "Pets": ["דוג מרקט"]}

# Reversing the dictionary above (created him that way in the first place because it's shorter to write)
sb2category = {}
for k,v in category2shortbiz.items():
    sb2category = {**sb2category,**{vi:k for vi in v}}

In [113]:
############################################################
### Creating an aggregation of payment sums per category ###
############################################################


# Defining a simple function to assign a category for each business name in the df and applying on the relevant column
def biz2category(biz_name, sb2c_dict):
  for k in sb2c_dict:
    if k in biz_name:
      return sb2c_dict[k]
  return 'Unknown Category'

expenses_df["Category"] = expenses_df["שם בית העסק"].apply(lambda x: biz2category(x, sb2category))

# Preprocessing the billing amount column to be able to sum it up
expenses_df["סכום החיוב"] = expenses_df["סכום החיוב"].apply(lambda x: x.replace("₪ ","").replace(",","")).astype(float)

# Finally, printing out the result
expenses_df.groupby("Category").sum("סכום החיוב")

,סכום החיוב
Category,
Bills,1184.45
Donations,434.80
Food and Groceris,1757.30
Internet Shopping,3841.05
Other,419.95
P2P payments,910.00
Pets,110.00
Transportation,554.93
